In [1]:
import os

import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    LSTM,
    Dense,
    Dropout,
    Flatten,
    Permute,
    Reshape,
    TimeDistributed,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Step 1: Data Preparation
# Load training images and labels
train_label_df = pd.read_csv(r"C:\Users\DELL\Downloads\data5\data\train\labels.csv")

# Define the path to the images folder
train_images_folder = r"C:\Users\DELL\Downloads\data5\data\train\images\images"

# List all image files in the images folder
image_files = os.listdir(train_images_folder)

# Filter out only the image files with valid extensions (e.g., jpg, png)
image_files = [
    file for file in image_files if file.endswith(".jpg") or file.endswith(".png")
]

# Extract the image names (without extensions) from the image file names
image_names = [os.path.splitext(file)[0] for file in image_files]

# Filter the label DataFrame to include only the labels for the images present in the images folder
train_label_df = train_label_df[train_label_df["name"].isin(image_names)]

# Define the train DataFrame with the appropriate data and labels
train_data = {
    "name": [
        os.path.join(train_images_folder, name + ".jpg")
        for name in train_label_df["name"]
    ],
    "text": train_label_df["text"],
}

train_df = pd.DataFrame(train_data)


# Text Preprocessing (Optional, uncomment if needed)
def preprocess_text(text):
    # Add your text normalization logic here (e.g., removing punctuation, handling diacritics)
    processed_text = text
    return processed_text


train_df["text"] = train_df["text"].apply(
    preprocess_text
)  # Apply preprocessing to text labels
import os

import numpy as np
import pandas as pd
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Step 2: Load and preprocess the images
# Define image dimensions
img_height, img_width = 224, 224  # Adjust as needed based on the chosen model

# Load and preprocess images using ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear intensity
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode="nearest"  # Fill mode for points outside the input boundaries
)

# Flow from DataFrame with image file paths and text labels
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="name",
    y_col="text",
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="categorical",  # Since we're using text labels, set class_mode to categorical
    shuffle=True,  # Shuffle the data
)

# Step 3: Build the model architecture
# Define the base model (pre-trained CNN model)
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the pre-trained layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Global average pooling layer
    Dropout(0.2),  # Dropout layer for regularization
    Dense(256, activation="relu"),  # Dense layer with ReLU activation
    Dropout(0.2),  # Additional dropout layer
    Dense(len(train_df["text"].unique()), activation="softmax")  # Output layer with softmax activation
])

# Step 4: Compile and train the model
# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Define early stopping callback
early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,  # Adjust as needed
    callbacks=[early_stopping],
)


Found 2508 validated image filenames belonging to 1638 classes.


16705208/16705208 [==============================] - 7s 0us/step

Epoch 1/20


79/79 [==============================] - 158s 2s/step - loss: 7.4185 - accuracy: 0.0140
Epoch 2/20
79/79 [==============================] - 136s 2s/step - loss: 7.3037 - accuracy: 0.0167
Epoch 3/20
79/79 [==============================] - 125s 2s/step - loss: 7.2322 - accuracy: 0.0175
Epoch 4/20
79/79 [==============================] - 128s 2s/step - loss: 7.1826 - accuracy: 0.0175
Epoch 5/20
79/79 [==============================] - 128s 2s/step - loss: 7.1685 - accuracy: 0.0179
Epoch 6/20
79/79 [==============================] - 131s 2s/step - loss: 7.1662 - accuracy: 0.0179
Epoch 7/20
79/79 [==============================] - 124s 2s/step - loss: 7.1655 - accuracy: 0.0179
Epoch 8/20
79/79 [==============================] - 124s 2s/step - loss: 7.1576 - accuracy: 0.0175
Epoch 9/20
79/79 [==============================] - 126s 2s/step - loss: 

In [10]:
import pandas as pd
from tensorflow.keras.preprocessing import image

# Step 1: Load the testing images
# Assuming you have a DataFrame or list containing paths to the testing images
testing_image_folder = r"C:\Users\DELL\Downloads\data5\data\evaluation\images\images"  # Adjust the path accordingly
testing_image_files = os.listdir(testing_image_folder)
testing_image_paths = [os.path.join(testing_image_folder, img) for img in testing_image_files]

# Step 2: Use your trained model to generate predictions on the testing images
def predict_labels(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Generate predictions
    predictions = model.predict(img_array)

    # Convert predictions to labels
    predicted_label = np.argmax(predictions)

    return predicted_label

# Apply the predict_labels function to each image path in the testing images
testing_image_predictions = [predict_labels(img_path) for img_path in testing_image_paths]

# Create a DataFrame with the results
submission_df = pd.DataFrame({
    "name": testing_image_files,
    "predicted_label": testing_image_predictions
})

# Now, you can save this DataFrame to a CSV file if needed or use it for further processing.





1/1 [==============================] - 0s 101ms/step


In [17]:
import pandas as pd
from tensorflow.keras.preprocessing import image

# Step 1: Load the testing images
# Assuming you have a DataFrame or list containing paths to the testing images
testing_image_folder = r"C:\Users\DELL\Downloads\data5\data\evaluation\images\images"  # Adjust the path accordingly
testing_image_files = os.listdir(testing_image_folder)
testing_image_paths = [os.path.join(testing_image_folder, img) for img in testing_image_files]

# Step 2: Use your trained model to generate predictions on the testing images
def predict_labels(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Generate predictions
    predictions = model.predict(img_array)

    # Convert predictions to labels
    predicted_label = np.argmax(predictions)

    return str(predicted_label)  # Convert the numeric label to a string

# Apply the predict_labels function to each image path in the testing images
testing_image_predictions = [predict_labels(img_path) for img_path in testing_image_paths]

# Create a dictionary to map numeric labels to unique Arabic labels
label_mapping = {str(label): f"Arabic_{label}" for label in set(testing_image_predictions)}

# Update the predict_labels function to use the label mapping
def predict_labels(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Generate predictions
    predictions = model.predict(img_array)

    # Convert predictions to labels
    predicted_label = np.argmax(predictions)

    return label_mapping[str(predicted_label)]  # Convert the numeric label to a string and use the label mapping

# Apply the predict_labels function to each image path in the testing images
testing_image_predictions = [predict_labels(img_path) for img_path in testing_image_paths]

# Create a DataFrame with the results
submission_df = pd.DataFrame({
    "name": testing_image_files,
    "text": testing_image_predictions
})



1/1 [==============================] - 0s 129ms/step


In [18]:
import pandas as pd

# Load the existing submission.csv file into a DataFrame
submission_df = pd.read_csv(r"C:\Users\DELL\Downloads\data5\data\evaluation\submission.csv")

# Convert the predicted labels to strings
predicted_labels_str = [str(label) for label in testing_image_predictions]

# Update the "text" column with the predicted labels as strings
submission_df["text"] = predicted_labels_str

# Save the updated DataFrame back to the submission.csv file
submission_df.to_csv("C:\\Users\\DELL\\Downloads\\submissionD.csv", index=False)




